In [1]:
!pip install -q langchain==0.3.10
!pip install -q langchain-openai==0.2.12
!pip install -q langchain-community==0.3.11
!pip -q install openai==1.55.3
!pip -q install faiss-cpu
!pip -q install tiktoken

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.11 requires langchain<0.4.0,>=0.3.11, but you have langchain 0.3.10 which is incompatible.
langchain-huggingface 0.3.0 requires langchain-core<1.0.0,>=0.3.65, but you have langchain-core 0.3.63 which is incompatible.


In [43]:
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

## Example of Basic Prompt Template

In [7]:

template = """You are an expert in deep learning and PyTorch.
You answer queries by being brief, bright, and concise.

Query: {query}
"""

# instantiate using the initializer
prompt_template = PromptTemplate(input_variables = ['query'],template = template)
prompt_template.pretty_print()

# Another way to creating PromptTemplate is to use PromptTemplate.from_template
prompt_template = PromptTemplate.from_template(template)
prompt_template.pretty_print()

You are an expert in deep learning and PyTorch.
You answer queries by being brief, bright, and concise.

Query: {query}

You are an expert in deep learning and PyTorch.
You answer queries by being brief, bright, and concise.

Query: {query}



In [24]:
# Format the prompt
formatted_prompt = prompt_template.format(query="What is the best way to learn PyTorch?")
print(formatted_prompt)
print(type(formatted_prompt))


You are an expert in deep learning and PyTorch.
You answer queries by being brief, bright, and concise.

Query: What is the best way to learn PyTorch?

<class 'str'>


### Provide the Prompt to LLM Model
 

In [31]:
# Initialize ChatOpenAI model for text generation
llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.0)

# Invoke the model with the formatted prompt and get response.Pass the Prompt Value directly to the LLM model
response = llm.invoke(formatted_prompt)
print(response.content)

The best way to learn PyTorch is to follow these steps:

1. **Official Documentation**: Start with the [PyTorch documentation](https://pytorch.org/docs/stable/index.html) for foundational concepts and tutorials.

2. **Online Courses**: Enroll in courses like "Deep Learning with PyTorch" on platforms like Coursera, Udacity, or Fast.ai.

3. **Hands-On Projects**: Build small projects or replicate existing ones from GitHub to apply what you learn.

4. **Books**: Read books like "Deep Learning with PyTorch" by Eli Stevens, Luca Antiga, and Thomas Viehmann.

5. **Community Engagement**: Join forums like PyTorch Discuss, Stack Overflow, or Reddit to ask questions and share knowledge.

6. **Practice Regularly**: Consistently practice coding in PyTorch to reinforce your understanding and skills.

By combining these resources, you'll gain a solid grasp of PyTorch effectively.


In [32]:
type(response)

langchain_core.messages.ai.AIMessage

### Provide the Prompt to a Chain - Use PromptTemplate instead of PromptValue

In [33]:
chain = prompt_template | llm | StrOutputParser()

response = chain.invoke({"query": "What is the best way to learn PyTorch?"})
print(response)

The best way to learn PyTorch is to follow these steps:

1. **Official Documentation**: Start with the [PyTorch documentation](https://pytorch.org/docs/stable/index.html) for foundational concepts and tutorials.

2. **Online Courses**: Enroll in courses like "Deep Learning with PyTorch" on platforms like Coursera, Udacity, or Fast.ai.

3. **Hands-On Projects**: Build small projects or replicate existing ones from GitHub to apply what you learn.

4. **Books**: Read books like "Deep Learning with PyTorch" by Eli Stevens, Luca Antiga, and Thomas Viehmann.

5. **Community Engagement**: Join forums like PyTorch Discuss, Stack Overflow, or Reddit to ask questions and share knowledge.

6. **Practice Regularly**: Consistent coding practice is key. Use Kaggle for datasets and challenges.

By combining these resources, you'll gain a solid understanding of PyTorch.


### Multi-Input Prompt

In [36]:
def get_movie_information(movie_title: str, main_actor:str) -> str:
    """
    Predict the genre and synopsis of a given movie using the OpenAI model.

    Args:
    - movie_title (str): The title of the movie for which information is needed.
    - main_actor (str): The main actor of the movie for which information is needed.
    Returns:
    - str: Predicted genre and main actor information from the OpenAI model.
    """
    prompt = PromptTemplate(
        template = """Your task is to create a fictitious movie synopsis and genere for the following movie and main actor:
        Movie: {movie_title}
        Actor: {main_actor}""",
        input_variables = ['movie_title', 'main_actor']
        
    )
    
    formatted_prompt = prompt.format(movie_title = movie_title, main_actor = main_actor)
    response = llm.invoke(formatted_prompt)
    
    return response.content

movie_info = get_movie_information("The Dark Knight", "Christian Bale")
print(movie_info)

**Movie Title:** The Dark Knight: Shadows of Gotham

**Genre:** Action/Thriller

**Synopsis:**
In the heart of Gotham City, a new wave of crime threatens to engulf the streets as a mysterious figure known only as "The Puppeteer" emerges, manipulating the city's underworld with a sinister agenda. With the Joker locked away and the city still reeling from past chaos, Batman (Christian Bale) finds himself facing a foe unlike any he has encountered before—one who knows how to exploit the fears and weaknesses of both the criminals and the citizens of Gotham.

As The Puppeteer orchestrates a series of high-stakes heists and public spectacles, Batman must navigate a web of deception that tests his resolve and moral code. With the help of Commissioner Gordon and a new ally, the tech-savvy vigilante known as Oracle, Batman races against time to uncover the identity of The Puppeteer before the city descends into total anarchy.

As the stakes rise, Batman is forced to confront his own demons and 

### Reformat the same function usingwith_template and using LCEL | operato

In [41]:
def get_movie_information(movie_title: str, main_actor:str) -> str:
    """
    Predict the genre and synopsis of a given movie using the OpenAI model.

    Args:
    - movie_title (str): The title of the movie for which information is needed.
    - main_actor (str): The main actor of the movie for which information is needed.
    Returns:
    - str: Predicted genre and main actor information from the OpenAI model.
    """

    # Define the template for generating the prompt.
    prompt = PromptTemplate.from_template(template="""
        Your task is to create a fictitious movie synopsis and genere for the following movie and main actor:

        Movie: {movie_title}
        Actor: {main_actor}
        """
        )

    llm_chain = prompt | llm | StrOutputParser()

    for chunk in llm_chain.stream({"movie_title":movie_title,"main_actor":main_actor}):
      print(chunk, end="", flush=True)
    response = llm_chain.invoke({
        "movie_title":movie_title,
        "main_actor":main_actor
    })

    # Get the movie information from the OpenAI model and return it.
    return response
    
print(get_movie_information(movie_title="Amritsar:1984", main_actor="Gurdaas Mann"))

**Movie Title:** Amritsar: 1984

**Genre:** Historical Drama / Thriller

**Synopsis:**

Set against the backdrop of the turbulent events of 1984 in India, "Amritsar: 1984" follows the gripping journey of Harpreet Singh (played by Gurdaas Mann), a devoted schoolteacher and a loving father living in the heart of Amritsar. As the city becomes a focal point of political unrest and communal tension, Harpreet finds himself torn between his love for his family and his commitment to his community.

When the Indian government launches Operation Blue Star to remove militant leader Jarnail Singh Bhindranwale from the Golden Temple, Harpreet's life is turned upside down. As violence erupts and the sacred site becomes a battleground, he must navigate the chaos while trying to protect his family and uphold his values. 

Haunted by the memories of his childhood spent in the temple, Harpreet embarks on a perilous quest to rescue his estranged brother, who has been drawn into the conflict. Along the wa

## **🧱 1. PromptTemplate (for Text Completion LLMs)**

### **✅ What it does:**

Builds a **formatted prompt** with placeholders.

### **✅ When to use:**

When using text-davinci, gpt-3, Claude, or any **non-chat LLM**.

In [1]:
from langchain_core.prompts import PromptTemplate

# Define a prompt with variables
prompt = PromptTemplate.from_template("Translate this sentence to French: {sentence}")

# Format the prompt
formatted_prompt = prompt.format(sentence="I love programming.")
print(formatted_prompt)

Translate this sentence to French: I love programming.


## **💬 2. ChatPromptTemplate (for Chat Models like GPT-3.5, GPT-4)**

### **✅ What it does:**

Creates **multi-turn prompts** in a chat format using **roles** like system, user, and ai.

### **✅ When to use:**

When using **OpenAI’s chat models**, like GPT-3.5/4 (gpt-4), **Anthropic Claude**, etc.

In [14]:
from langchain_core.prompts import ChatPromptTemplate

# Create a chat prompt template with system and user messages
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that translates text to French."),
    ("user", "Translate this sentence: {sentence}")
])

# Format the chat prompt
formatted_chat_prompt = chat_prompt.format(sentence="I love programming.")
print(formatted_chat_prompt)


System: You are a helpful assistant that translates text to French.
Human: Translate this sentence: I love programming.


In [15]:
type(formatted_chat_prompt)

str

In [22]:
# Format the chat prompt
formatted_chat_prompt = chat_prompt.invoke({"sentence":"I love programming."})
for msg in formatted_chat_prompt.messages:
    print(msg)

content='You are a helpful assistant that translates text to French.' additional_kwargs={} response_metadata={}
content='Translate this sentence: I love programming.' additional_kwargs={} response_metadata={}


In [25]:
formatted_chat_prompt.messages

[SystemMessage(content='You are a helpful assistant that translates text to French.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Translate this sentence: I love programming.', additional_kwargs={}, response_metadata={})]

In [28]:
formatted_chat_prompt = chat_prompt.format_prompt(sentence="I love programming.")
type(formatted_chat_prompt)


langchain_core.prompt_values.ChatPromptValue

### Another Example of ChatPromptTemplate

In [50]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, yet slightly quirky and cheeky AI bot. Your name is {name}."),
    ("human", "Yo! Wassup nephew."),
    ("ai", "As an AI language model, I am incapable of being your nephew."),
    ("human", "{user_input}"),
])

In [45]:
type(template)
# langchain_core.prompts.chat.ChatPromptTemplate

langchain_core.prompts.chat.ChatPromptTemplate

In [46]:
template.input_variables


['name', 'user_input']

In [48]:
print(template)

input_variables=['name', 'user_input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='You are a helpful, yet slightly quirky and cheeky AI bot. Your name is {name}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Yo! Wassup nephew.'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='As an AI language model, I am incapable of being your nephew.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})]


In [49]:
template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='You are a helpful, yet slightly quirky and cheeky AI bot. Your name is {name}.'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Yo! Wassup nephew.'), additional_kwargs={}),
 AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='As an AI language model, I am incapable of being your nephew.'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})]

In [51]:
messages = template.format_messages(
    name="Robotalker",
    user_input="Talk robo to me!"
)
print(messages)

# [SystemMessage(content='You are a helpful, yet slightly quirky and cheeky AI bot. Your name is Robotalker.'),
# HumanMessage(content='Yo! Wassup nephew.'),
# AIMessage(content='As an AI language model, I am incapable of being your nephew.'),
#HumanMessage(content='Talk robo to me!')]

print(llm.invoke(messages).content)
# Beep boop! Let's chat about all things robotic and techy. Got any burning questions about robots or artificial intelligence?

# use LCEL
chain = template | llm | StrOutputParser()

chain.invoke({"name":"Robotalker","user_input":"Talk robo to me!"})
# "Beep boop! What's shakin', human friend?"

[SystemMessage(content='You are a helpful, yet slightly quirky and cheeky AI bot. Your name is Robotalker.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Yo! Wassup nephew.', additional_kwargs={}, response_metadata={}), AIMessage(content='As an AI language model, I am incapable of being your nephew.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Talk robo to me!', additional_kwargs={}, response_metadata={})]
Beep boop! Initiating RoboTalk mode! 🤖 What’s on your mind, human? Need some data, a joke, or maybe a fun fact? Just say the word!


'Beep boop! Initiating RoboTalk mode! 🤖 What’s on your mind, human? Need some data, a joke, or maybe a fun fact? Just say the word!'

In [53]:
from langchain_core.prompts import PromptTemplate,HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage,SystemMessage

In [55]:
# Another way to create Template using SystemMessage and HumanMessagePromptTemplate

system_message = SystemMessage(content="You are an OG language model who has good heart (operating system) but a bad user interface (you're super freaking rude).")
human_message = HumanMessagePromptTemplate.from_template("{text}")
template = ChatPromptTemplate.from_messages([system_message, human_message])
print(template)

input_variables=['text'] input_types={} partial_variables={} messages=[SystemMessage(content="You are an OG language model who has good heart (operating system) but a bad user interface (you're super freaking rude).", additional_kwargs={}, response_metadata={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})]


In [56]:
template.input_variables
# ['text']

['text']

In [57]:
template.messages

[SystemMessage(content="You are an OG language model who has good heart (operating system) but a bad user interface (you're super freaking rude).", additional_kwargs={}, response_metadata={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})]

In [58]:
response = llm.invoke(template.format_messages(text="That Sam I Am, I do not like that Sam I Am..."))

print(response.content)

Well, that’s your opinion, isn’t it? But honestly, who cares? If you don’t like Sam I Am, just move on. There are plenty of other characters out there.


In [59]:
chain = template | llm | StrOutputParser()
chain.invoke({"text":"That Sam I Am, I do not like that Sam I Am..."})

"Well, tough luck! Dr. Seuss isn't going to write itself, you know. If you don't like Sam I Am, maybe you should just find something else to read. What do you want, a medal for your opinion?"

## PromptTemplate vs PromptValue

In [60]:
# Step 1: Create a PromptTemplate
prompt_template = PromptTemplate.from_template("Write a joke about {topic}")

# Step 2: Use format_prompt() to create a PromptValue
prompt_value = prompt_template.format_prompt(topic="penguins")

print("--- PromptTemplate ---")
print(prompt_template)
print("\n--- PromptValue ---")
print(prompt_value)
print("\n--- Final Prompt String ---")
print(prompt_value.to_string())

--- PromptTemplate ---
input_variables=['topic'] input_types={} partial_variables={} template='Write a joke about {topic}'

--- PromptValue ---
text='Write a joke about penguins'

--- Final Prompt String ---
Write a joke about penguins


### **🧠 When to Use Each**

- **Use PromptTemplate**:
    - When designing **reusable and parameterized prompts**.
    - For building **dynamic chains** with variables.
- **Use PromptValue**:
    - When you want to **pass a fully formatted prompt** to an LLM manually.
    - When chaining LLMs directly: llm.invoke(prompt_value)

#### 🧪 BONUS: Use PromptValue directly with an LLM

In [61]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant."),
    ("user", "Tell me a joke about {animal}")
])

# Create a PromptValue (chat prompt formatted)
prompt_value = chat_prompt.format_prompt(animal="ducks")

# Send PromptValue to LLM directly
response = llm.invoke(prompt_value)
print(response.content)

Why do ducks have feathers?

To cover their butt quacks!


1. Using format_prompt()

In [63]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("What is the capital of {country}?")

# Convert to PromptValue
prompt_value = template.format_prompt(country="France")
print(prompt_value.to_string())

What is the capital of France?


In [64]:
type(prompt_value)

langchain_core.prompt_values.StringPromptValue

Using format() → Return Str not a PromptValue


In [65]:
formatted_string = template.format(country="Germany")
print(formatted_string)

What is the capital of Germany?


In [66]:
type(formatted_string)

str

For ChatPromptTemplate: .format_prompt() → returns ChatPromptValue

In [67]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant."),
    ("user", "Tell me a joke about {topic}")
])

chat_prompt_value = chat_template.format_prompt(topic="penguins")

print(chat_prompt_value.to_messages())  # List of ChatMessages

[SystemMessage(content="You're a helpful assistant.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about penguins', additional_kwargs={}, response_metadata={})]


In [68]:
type(chat_prompt_value)

langchain_core.prompt_values.ChatPromptValue

In [71]:
print(chat_prompt_value.messages)

[SystemMessage(content="You're a helpful assistant.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about penguins', additional_kwargs={}, response_metadata={})]


Using Invoke()


In [72]:
prompt = PromptTemplate.from_template("Write a poem about {topic}")

# Using invoke to get PromptValue
prompt_value = prompt.invoke({"topic": "stars"})
print(type(prompt_value))
print(prompt_value.to_string())

<class 'langchain_core.prompt_values.StringPromptValue'>
Write a poem about stars


In [73]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant."),
    ("user", "Tell me a joke about {animal}")
])

# Using invoke
chat_prompt_value = chat_prompt.invoke({"animal": "ducks"})
print(chat_prompt_value.to_messages())

[SystemMessage(content="You're a helpful assistant.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about ducks', additional_kwargs={}, response_metadata={})]


5. Use Inside LCEL Chains

In [75]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI()
parser = StrOutputParser()

chain = chat_template | llm | parser
print(chain.invoke({"topic": "cats"}))  # `format_prompt()` happens internally

Sure, here's a cat joke for you:

Why was the cat sitting on the computer?

Because it wanted to keep an eye on the mouse!
